<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#类型示例和聚类" data-toc-modified-id="类型示例和聚类-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>类型示例和聚类</a></span></li><li><span><a href="#k-means聚类" data-toc-modified-id="k-means聚类-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>k-means聚类</a></span></li><li><span><a href="#实际案例：给动物们分类" data-toc-modified-id="实际案例：给动物们分类-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>实际案例：给动物们分类</a></span></li><li><span><a href="#结论：" data-toc-modified-id="结论：-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>结论：</a></span></li></ul></div>

In [76]:
import pylab
import random

In [77]:
#闵可夫斯基距离度量函数
def minknowskiDist(v1, v2, p):
    dist = 0.0
    for i in range(len(v1)):
        dist += abs(v1[i] - v2[i]) ** p

    return dist ** (1.0 / p)

# 类型示例和聚类

In [78]:
#用Example来构建聚类实例，每个实例有一个名字、一个特征向量和一个可选的标签，distance方法返回两个实例之间的欧氏距离。
class Example(object):
    
    def __init__(self, name, features, label = None):
        #假设特征值为一个数值序列
        self.name = name
        self.features = features
        self.label = label
        
    def dimensionality(self):#返回特征向量长度
        return len(self.features)
    
    def getFeatures(self): #返回特征向量值
        return self.features[:]
    
    def getLabel(self): #返回标签
        return self.label
    
    def getName(self): #返回实例名字
        return self.name
    
    def distance(self, other): #返回两个实例的欧式距离
        return minknowskiDist(self.features, other.getFeatures(), 2)
    
    def __str__(self):
        return self.name +':'+ str(self.features) + ':' + str(self.label)

In [79]:
"""
一个簇就是一组实例，质心（centroid）可以看作是簇的质心，computeCentroid方法会返回质心实例。
它的特征向量等于簇中所有实例特征向量的欧式平均值
"""

class Cluster(object):
    
    def __init__(self, examples, exampleType):
        """Assumes examples is a list of example of type exampleType"""
        self.examples = examples
        self.exampleType = exampleType
        self.centroid = self.computeCentroid()#质心的特征向量通过computeCentroid计算
        
    def update(self, examples):
        """Replace the examples in the cluster by new examples
           Return how much the centroid has changed"""
        oldCentroid = self.centroid
        self.examples = examples
        if len(examples) > 0:
            self.centroid = self.computeCentroid()
            return oldCentroid.distance(self.centroid)
        else:
            return 0.0
        
    def members(self):#簇中实例举例
        for e in self.examples:
            yield e
        
    def size(self):#实例表大小
        return len(self.examples)
    
    def getCentroid(self):
        return self.centroid
    
    def computeCentroid(self):
        dim = self.examples[0].dimensionality()
        totVals = pylab.array([0.0]*dim)
        for e in self.examples:
            totVals += e.getFeatures()
        centroid = self.exampleType('centroid',
                              totVals/float(len(self.examples)))#所有特征向量的欧式平均值
        return centroid
    
    def variance(self):#本簇方差
        totDist = 0.0
        for e in self.examples:
            totDist += (e.distance(self.centroid))**2
        return totDist**0.5
    
    def __str__(self):
        names = []
        for e in self.examples:
            names.append(e.getName())
        names.sort()
        result = 'Cluster with centroid '\
                 + str(self.centroid.getFeatures()) + ' contains:\n  '
        for e in names:
            result = result + e + ', '
        return result[:-2]

# k-means聚类

In [80]:
"""
聚类方式有分层聚类、EM聚类和K-means等
K-means聚类，可能是最常用的聚类方法，目标是把一组实例分割成K个簇，满足：
（1）每个实例都处在质心距离实例质心最近的簇中，并且
（2）所有簇的差异总和最小。

算法（贪心算法）：
1.随机选择K个实例作为初始质量心
2.无线循环（while true）
    （1）每个实例分配给最近的质量心，从而创建k个簇
    （2）通过平均每个簇的实例来得到k个新质心
    （3）若所有质心都和计算之前相同，返回当前的簇集合
"""
def kmeans(examples, exampleType, k, verbose):
    """Assumes examples is a list of examples of type exampleType,
         k is a positive int, verbose is a Boolean
       Returns a list containing k clusters. If verbose is
         True it prints result of each iteration of k-means"""
    #Get k randomly chosen initial centroids
    initialCentroids = random.sample(examples, k)
    
    #Create a singleton cluster for each centroid
    clusters = []
    for e in initialCentroids:
        clusters.append(Cluster([e], exampleType))
        
    #Iterate until centroids do not change
    converged = False
    numIterations = 0
    while not converged:
        numIterations += 1
        #Create a list containing k distinct empty lists
        newClusters = []
        for i in range(k):
            newClusters.append([])

        #Associate each example with closest centroid
        for e in examples:
            #Find the centroid closest to e
            smallestDistance = e.distance(clusters[0].getCentroid())
            index = 0
            for i in range(1, k):
                distance = e.distance(clusters[i].getCentroid())
                if distance < smallestDistance:
                    smallestDistance = distance
                    index = i
            #Add e to the list of examples for the appropriate cluster
            newClusters[index].append(e)
            
        #Upate each cluster; check if a centroid has changed
        converged = True
        for i in range(len(clusters)):
            if clusters[i].update(newClusters[i]) > 0.0:
                converged = False
        if verbose:
            print ('Iteration #' + str(numIterations))
            for c in clusters:
                print (c)
            print ('') #add blank line
    return clusters

#计算一组簇的方差和
def dissimilarity(clusters):
    totDist = 0.0
    for c in clusters:
        totDist += c.variance()
    return totDist

In [81]:
#寻找最佳K-means聚类
def trykmeans(examples, exampleType, numClusters, numTrials,
              verbose = False):
    """Calls kmeans numTrials times and returns the result with the
          lowest dissimilarity"""
    best = kmeans(examples, exampleType, numClusters, verbose)
    minDissimilarity = dissimilarity(best)
    for trial in range(1, numTrials):
        clusters = kmeans(examples, exampleType, numClusters, verbose)
        currDissimilarity = dissimilarity(clusters)
        if currDissimilarity < minDissimilarity:
            best = clusters
            minDissimilarity = currDissimilarity
    return best

# 实际案例：给动物们分类

In [82]:
"""
该函数会读入示例文件并生成一组实例。
首先处理文件开头信息，从而得到每个实例的特征数量；然后会使用下面的内容生成三个列表：
（1）speciesNames包含所有哺乳动物的名称
（2）labelList包含每个动物对应的标签（食草、食肉和杂食）
（3）featureVals中的每个元素都是一个列表，这个列表包含所有哺乳动物在某个特征上的值。
    featuresVals[i][j]表示第j个哺乳动物第i个特征值。
"""
def readMammalData(fName):
    dataFile = open(fName, 'r')
    numFeatures = 0
    #Process lines at top of file
    for line in dataFile: #Find number of features
        if line[0:6] == '#Label': #indicates end of features
            break
        if line[0:5] != '#Name':
            numFeatures += 1
    featureVals = []
    
    #Produce featureVals, speciesNames, and labelList
    featureVals, speciesNames, labelList = [], [], []
    for i in range(numFeatures):
        featureVals.append([])
        
    #Continue processing lines in file, starting after comments
    for line in dataFile:
#         dataLine = string.split(line[:-1], ',') #remove newline; then split
        dataLine=line[:-1].split(',')
        speciesNames.append(dataLine[0])
        classLabel = float(dataLine[-1])
        labelList.append(classLabel)
        for i in range(numFeatures):
            featureVals[i].append(float(dataLine[i+1]))
            
    #Use featureVals to build list containing the feature vectors
    #for each mammal
    featureVectorList = []
    for mammal in range(len(speciesNames)):
        featureVector = []
        for feature in range(numFeatures):
            featureVector.append(featureVals[feature][mammal])
        featureVectorList.append(featureVector)
    return featureVectorList, labelList, speciesNames

In [86]:
"""函数testTeeth会使用trymeans来聚类
   buildMammalExamples生成的实例，接着会输出每个簇中食草、食肉和杂食动物的数量"""
def buildMammalExamples(featureList, labelList, speciesNames):
    examples = []
    for i in range(len(speciesNames)):
        features = pylab.array(featureList[i])
        example = Example(speciesNames[i], features, labelList[i])
        examples.append(example)
    return examples

def testTeeth(numClusters, numTrials):
    features, labels, species = readMammalData('dentalFormulas.txt')
    examples = buildMammalExamples(features, labels, species)
    bestClustering =\
                   trykmeans(examples, Example, numClusters, numTrials)
    for c in bestClustering:
        names = ''
        for p in c.members():
            names += p.getName() + ', '
        print ('\n', names[:-2]) #remove trailing comma and space
        herbivores, carnivores, omnivores = 0, 0, 0
        for p in c.members():
            if p.getLabel() == 0:
                herbivores += 1
            elif p.getLabel() == 1:
                carnivores += 1
            else:
                omnivores += 1
        print (herbivores, '食草动物,', carnivores, '食肉动物,',\
              omnivores, '杂食动物')

In [89]:
testTeeth(3,20)


 獾, 美洲狮, 狗, 狐狸, 豚鼠, 人, 美洲虎, 袋鼠, 貂, 鼹鼠, 老鼠, 豪猪, 猪, 兔子, 浣熊, 鼠, 红蝙蝠, 臭鼬, 松鼠, 土拨鼠, 狼
4 食草动物, 9 食肉动物, 8 杂食动物

 熊, 牛, 鹿, 海狗, 海豹, 麋鹿, 狮子, 海狮
3 食草动物, 4 食肉动物, 1 杂食动物

 驼鹿
1 食草动物, 0 食肉动物, 0 杂食动物


# 聚类受到了部分特征向量取值范围过大的影响

# 特征值标准化

In [90]:
#特征调整函数，让每个特征的平均值为0，标准差为1。
def scaleFeatures(vals):
    """Assumes vals is a sequence of numbers"""
    result = pylab.array(vals)
    mean = sum(result)/float(len(result))
    result = result - mean
    sd = stdDev(result)
    result = result/sd
    return result

In [91]:
def stdDev(X):
    mean = float(sum(X))/len(X)
    tot = 0.0
    for x in X:
        tot += (x-mean)**2
    return (tot/len(X))**0.5

In [92]:
v1, v2 = [], []
for i in range(1000):
    v1.append(random.gauss(100, 5))
    v2.append(random.gauss(50, 10))
print ('v1 mean =', round(sum(v1)/len(v1), 4),\
      'v1 standard deviation', round(stdDev(v1), 4))
print ('v2 mean =', round(sum(v2)/len(v2), 4),\
      'v1 standard deviation', round(stdDev(v2), 4))  
v1 = scaleFeatures(v1)
v2 = scaleFeatures(v2)
print ('v1 mean =', round(sum(v1)/len(v1), 4),\
      'v1 standard deviation', round(stdDev(v1), 4))
print ('v2 mean =', round(sum(v2)/len(v2), 4),\
      'v1 standard deviation', round(stdDev(v2), 4))

v1 mean = 99.944 v1 standard deviation 4.8953
v2 mean = 50.1868 v1 standard deviation 10.1379
v1 mean = -0.0 v1 standard deviation 1.0
v2 mean = -0.0 v1 standard deviation 1.0


In [93]:
def readMammalData(fName, scale):
    """Assumes scale is a Boolean.  If True, features are scaled"""
    dataFile = open(fName, 'r')
    numFeatures = 0
    #Process lines at top of file
    for line in dataFile: #Find number of features
        if line[0:6] == '#Label': #indicates end of features
            break
        if line[0:5] != '#名称':
            numFeatures += 1
    featureVals = []
    
    #Produce featureVals, speciesNames, and labelList
    featureVals, speciesNames, labelList = [], [], []
    for i in range(numFeatures):
        featureVals.append([])
        
    #Continue processing lines in file, starting after comments
    for line in dataFile:
        dataLine=line[:-1].split(',')
        
        speciesNames.append(dataLine[0])
        classLabel = float(dataLine[-1])
        labelList.append(classLabel)
        for i in range(numFeatures):
            featureVals[i].append(float(dataLine[i+1]))
            
    #Use featureVals to build list containing the feature vectors
    #for each mammal scale features, if needed
    if scale:
        for i in range(numFeatures):
            featureVals[i] = scaleFeatures(featureVals[i])
    featureVectorList = []
    for mammal in range(len(speciesNames)):
        featureVector = []
        for feature in range(numFeatures):
            featureVector.append(featureVals[feature][mammal])
        featureVectorList.append(featureVector)
    return featureVectorList, labelList, speciesNames

In [100]:
def testTeeth(numClusters, numTrials, scale):
    features,classes,species=\
            readMammalData('dentalFormulas.txt',scale)
    examples=buildMammalExamples(features,classes,species)
    bestClustering =\
                   trykmeans(examples, Example, numClusters, numTrials)
    for c in bestClustering:
        names = ''
        for p in c.members():
            names += p.getName() + ', '
        print ('\n', names[:-2]) #remove trailing comma and space
        herbivores, carnivores, omnivores = 0, 0, 0
        for p in c.members():
            if p.getLabel() == 0:
                herbivores += 1
            elif p.getLabel() == 1:
                carnivores += 1
            else:
                omnivores += 1
        print (herbivores, '食草动物,', carnivores, '食肉动物,',\
              omnivores, '杂食动物')
    

In [101]:
print('Cluster without scaleing')
testTeeth(3,20,False)

Cluster without scaleing

 驼鹿
1 食草动物, 0 食肉动物, 0 杂食动物

 獾, 美洲狮, 鹿, 狗, 狐狸, 海狗, 豚鼠, 人, 美洲虎, 袋鼠, 狮子, 貂, 鼹鼠, 老鼠, 豪猪, 猪, 兔子, 浣熊, 鼠, 红蝙蝠, 臭鼬, 松鼠, 土拨鼠, 狼
5 食草动物, 11 食肉动物, 8 杂食动物

 熊, 牛, 海豹, 麋鹿, 海狮
2 食草动物, 2 食肉动物, 1 杂食动物


In [106]:
print('Cluster with scaleing')
testTeeth(3,20,True)

Cluster with scaleing

 獾, 熊, 美洲狮, 狗, 狐狸, 海狗, 海豹, 麋鹿, 人, 美洲虎, 狮子, 貂, 鼹鼠, 猪, 浣熊, 红蝙蝠, 海狮, 臭鼬, 狼
1 食草动物, 13 食肉动物, 5 杂食动物

 牛, 鹿, 豚鼠, 驼鹿, 老鼠, 豪猪, 兔子, 鼠, 松鼠, 土拨鼠
6 食草动物, 0 食肉动物, 4 杂食动物

 袋鼠
1 食草动物, 0 食肉动物, 0 杂食动物


# 结论：
    理想结果是完全分为三类（食草、食肉、杂食），但从上述结果看出仅仅基于齿类信息，不足以完善杂食动物的分类规律，或许应当加入牙列和重量以外的特征值对杂食动物进行进一步区分。